In [1]:
import ftplib
import ssl

class ImplicitFTP_TLS(ftplib.FTP_TLS):
    """FTP_TLS subclass that automatically wraps sockets in SSL to support implicit FTPS."""

    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self._sock = None

    @property
    def sock(self):
        """Return the socket."""
        return self._sock

    @sock.setter
    def sock(self, value):
        """When modifying the socket, ensure that it is ssl wrapped."""
        if value is not None and not isinstance(value, ssl.SSLSocket):
            value = self.context.wrap_socket(value)
        self._sock = value

In [68]:
ftp_client = ImplicitFTP_TLS()
ftp_client.connect(host='187.111.98.100', port=990)
ftp_client.login(user='FTP_SMTR_SMTR', passwd='USR@R1oocAR#30')
ftp_client.prot_p()

'200 Protection level set to P'

In [18]:
ftp_client.cwd('APROCESSAR')

'250 CWD successful. "/APROCESSAR" is current directory.'

In [73]:
from dateutil import parser
sys.path.append("/home/lmoraes/maestro/")
from repositories.helpers.helpers import read_config
from pathlib import Path
import re
RDO_DIRECTORY = os.getenv("RDO_DATA_FTPS", "/home/lmoraes/RDO_DATA_FTPS")
import jinja2

In [22]:
for filepath in ftp_client.mlsd():
    name = filepath[0]
    timestamp = filepath[1]['modify']
    time = parser.parse(timestamp)
    print(name + ' - ' + str(time))
    break


RDO2_00153_00001_20170514_20170515_234344.TXT - 2017-05-16 14:56:29


In [74]:
ftp_client.cwd('/')
for folder in ftp_client.mlsd():
    # Config yaml file will be folder_fileprefix.yaml
    if folder[1]['type'] == 'dir':
        # CWD to folder
        print(f"Entering folder {folder[0]}")
        ftp_client.cwd(folder[0])
        folder_name = folder[0].lower()

        # Read file list
        for filepath in ftp_client.mlsd():
            filename = filepath[0]
            fileprefix = filename.split('_')[0].lower()
            timestamp = filepath[1]['modify']
            time = parser.parse(timestamp)

            # Download file to local folder
            try:
                # config = read_config(Path(__file__).parent / f'br_rj_riodejaneiro_rdo/{folder}_{fileprefix}.yaml') 
                
                config_file = f'/home/lmoraes/maestro/repositories/capturas/br_rj_riodejaneiro_rdo/{folder_name}_{fileprefix}.yaml'
                config = read_config(Path(config_file)) 
                table_id = config['resources']['basedosdados_config']['config']['table_id']
                date = tuple(re.findall("\d+", filename))
                ano = date[2][:4]
                mes = date[2][4:6]
                dia = date[2][6:]
                relative_filepath = Path('raw/br_rj_riodejaneiro_rdo', table_id, f'ano={ano}', f'mes={mes}', f'dia={dia}')
                local_filepath = Path(RDO_DIRECTORY, relative_filepath)
                Path(local_filepath).mkdir(parents=True, exist_ok=True)
                with open(f'{local_filepath}/{filename}', 'wb') as local_file:
                    ftp_client.retrbinary('RETR ' + filename, local_file.write)
                
                # Run pipeline
                config['solids']['parse_file_path_and_partitions']['inputs']['bucket_path']['value'] = f'{relative_filepath}/{filename}'
                config['solids']['upload_file_to_storage'] = {"inputs": {"file_path": {"value": f'{local_filepath}/{filename}'}}}
                print(config)
            except jinja2.TemplateNotFound as err:
                print(err)
                print(f"Config file {config_file} was not found. Skipping file.")
            
        ftp_client.cwd('/')
    else:
        print(f"Skipping file {folder[0]} since it is not inside a folder")
        continue

l', 'gratuidade_estudante_estadual', 'gratuidade_estudante_municipal', 'universitario', 'gratuito_rodoviario', 'buc_1a_perna', 'buc_2a_perna', 'buc_receita', 'buc_supervia_1a_perna', 'buc_supervia_2a_perna', 'buc_supervia_receita', 'buc_van_1a_perna', 'buc_van_2a_perna', 'buc_van_receita', 'buc_vlt_1a_perna', 'buc_vlt_2a_perna', 'buc_vlt_receita', 'buc_brt_1a_perna', 'buc_brt_2a_perna', 'buc_brt_3a_perna', 'buc_brt_receita', 'buc_inter_1a_perna', 'buc_inter_2a_perna', 'buc_inter_receita', 'buc_barcas_1a_perna', 'buc_barcas_2a_perna', 'buc_barcas_receita', 'buc_metro_1a_perna', 'buc_metro_2a_perna', 'buc_metro_receita', 'cartao', 'receita_cartao', 'especie_passageiro_transportado', 'especie_receita', 'registro_processado', 'data_processamento', 'linha_rcti'], 'index_col': False}, 'reindex_kwargs': {'columns': ['operadora', 'linha', 'data_transacao', 'tarifa_valor', 'gratuidade_idoso', 'gratuidade_especial', 'gratuidade_estudante_federal', 'gratuidade_estudante_estadual', 'gratuidade_est

In [28]:
ftp_client.cwd('/')
for folder in ftp_client.mlsd():
    # print(folder)
    # Config yaml file will be folder_fileprefix.yaml
    if folder[1]['type'] == 'dir':
        # CWD to folder
        print(f"Entering folder {folder[0]}")
        ftp_client.cwd(folder[0])
        ftp_client.cwd('/')
    else:
        print(f"Skipping file {folder[0]} since it is not inside a folder")
        continue

Entering folder APROCESSAR
Skipping file RDO_00700_00001_20210508_20210513_050941.TXT since it is not inside a folder
Entering folder SPPO
Entering folder STPL


In [84]:
 fstats = os.stat('ftps.ipynb')
 fstats

os.stat_result(st_mode=33188, st_ino=212082, st_dev=2064, st_nlink=1, st_uid=1000, st_gid=1000, st_size=29010, st_atime=1621877682, st_mtime=1621886391, st_ctime=1621886391)

In [79]:
datetime.timestamp(time)

1621849611.0

In [86]:
time

datetime.datetime(2021, 5, 24, 6, 46, 51)

In [85]:
datetime.fromtimestamp(1621886391)

datetime.datetime(2021, 5, 24, 16, 59, 51)

In [78]:
from datetime import datetime